In [ ]:
#from https://towardsdatascience.com/the-lesser-known-rising-application-of-llms-775834116477

!git clone https://github.com/shawngraham/structured-cultural-heritage-crime

In [3]:
%cd structured-cultural-heritage-crime

/content/structured-cultural-heritage-crime


In [ ]:
!pip install -r requirements.txt
#should remove mistral and replace with openai


In [6]:
!pip install python-dotenv

In [ ]:
!pip install langchain-openai


In [6]:
import os
import json
import time
from pathlib import Path

import pandas as pd
from langchain.output_parsers import PydanticOutputParser
#from langchain_mistralai.chat_models import ChatMistralAI
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

from core import run
from prompt import DEFAULT_BASE_PROMPT, create_prompt
from schemas import CulturalHeritageSchema


In [7]:

from google.colab import userdata
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

In [9]:
!unzip ../texts.zip

Archive:  ../texts.zip
   creating: texts/
  inflating: texts/true.txt          
  inflating: __MACOSX/texts/._true.txt  
  inflating: texts/kouros.txt        
  inflating: __MACOSX/texts/._kouros.txt  
  inflating: texts/medici.txt        
  inflating: __MACOSX/texts/._medici.txt  


In [10]:
import os
import pandas as pd

def read_files_to_dataframe(folder_path: str) -> pd.DataFrame:
    # List all text files in the folder
    file_names = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

    # Read each file's content into a list
    data = []
    for file_name in file_names:
        with open(os.path.join(folder_path, file_name), 'r', encoding='utf-8') as file:
            content = file.read()
            data.append(content)

    # Create a DataFrame with the file contents
    df = pd.DataFrame(data, columns=['content'])

    return df

# Usage example
folder_path = 'texts'
df = read_files_to_dataframe(folder_path)
print(df)

                                             content
0  Marion True was Curator of Antiquities at the ...
1  The Getty kouros (youth) is a 2 m high marble ...
2  Giacomo Medici is an Italian antiquities deale...


In [74]:
import importlib
import sys
importlib.reload(sys.modules['prompt'])
importlib.reload(sys.modules['schemas'])
from prompt import DEFAULT_BASE_PROMPT, create_prompt
from schemas import CulturalHeritageSchema

model_name = "gpt-4o"
temperature = 0
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model_name=model_name, temperature=temperature)
parser = PydanticOutputParser(pydantic_object=CulturalHeritageSchema)
prompt = create_prompt(DEFAULT_BASE_PROMPT, parser, df["content"][0])
prompt


[HumanMessage(content='\nExtract information regarding cultural heritage crime from the following text.\n\nIn particular, please provide the following information:\n- A list of triplets, where each triplet consists of two entities and the relation between them. Only use the predefined relations listed below and provide detailed descriptions for each entity.\n\nThe predefined relations are:\n- \'is_the_owner_of\': Denotes a business relationship where an actor controls, or is the legal owner of, a business, gallery, auction house, or other for-profit organization.\n- \'works_with\': Denotes a business relationship between actors who are dealers, organizations, looters, or collectors.\n- \'works_for\': Describes an employment or contractual relationship between actors who are dealers, organizations, businesses, museums, government agencies, looters, or collectors.\n- \'has_possession_of\': Describes a situation where a dealer, organization, collector, or auction house controls an artefac

In [75]:
example = await run(llm, prompt, parser)
example

CulturalHeritageSchema(triplets=[Triplet(entity1=Entity(name='Marion True'), relation=Relation(name=<RelationType.works_for: 'works_for'>), entity2=Entity(name='J. Paul Getty Museum')), Triplet(entity1=Entity(name='Jiri Frel'), relation=Relation(name=<RelationType.works_for: 'works_for'>), entity2=Entity(name='J. Paul Getty Museum')), Triplet(entity1=Entity(name='Marion True'), relation=Relation(name=<RelationType.has_role: 'has_role'>), entity2=Entity(name='Curator of Antiquities')), Triplet(entity1=Entity(name='Jiri Frel'), relation=Relation(name=<RelationType.has_role: 'has_role'>), entity2=Entity(name='Curator of Antiquities')), Triplet(entity1=Entity(name='J. Paul Getty Museum'), relation=Relation(name=<RelationType.has_possession_of: 'has_possession_of'>), entity2=Entity(name='Getty Aphrodite')), Triplet(entity1=Entity(name='J. Paul Getty Museum'), relation=Relation(name=<RelationType.purchases: 'purchases'>), entity2=Entity(name='Getty Aphrodite')), Triplet(entity1=Entity(name='

In [76]:
## on all the text
raw_outputs = []
outputs = []

for i in range(df.shape[0]):

    prompt = create_prompt(DEFAULT_BASE_PROMPT, parser, df["content"][i])
    result = await run(llm, prompt, parser)
    raw_outputs.append(result)

    output_map = {
        "article_id":i,
    }

    if isinstance(result, CulturalHeritageSchema):
        output =  output_map | result.dict()
        outputs.append(output)
    else:
        output_map["raw_llm_output"] = result
        outputs.append(output_map)

    time.sleep(1)

Error in parsing Failed to parse CulturalHeritageSchema from completion {"triplets": [{"entity1": {"name": "Giacomo Medici"}, "relation": {"name": "is_the_owner_of"}, "entity2": {"name": "Antiquaria Romana"}}, {"entity1": {"name": "Giacomo Medici"}, "relation": {"name": "is_the_owner_of"}, "entity2": {"name": "Editions Services"}}, {"entity1": {"name": "Giacomo Medici"}, "relation": {"name": "works_with"}, "entity2": {"name": "Robert Hecht"}}, {"entity1": {"name": "Giacomo Medici"}, "relation": {"name": "works_with"}, "entity2": {"name": "Christian Boursaud"}}, {"entity1": {"name": "Giacomo Medici"}, "relation": {"name": "works_with"}, "entity2": {"name": "Robin Symes"}}, {"entity1": {"name": "Giacomo Medici"}, "relation": {"name": "works_with"}, "entity2": {"name": "Frieda Tchacos"}}, {"entity1": {"name": "Giacomo Medici"}, "relation": {"name": "works_with"}, "entity2": {"name": "Nikolas Koutoulakis"}}, {"entity1": {"name": "Giacomo Medici"}, "relation": {"name": "works_with"}, "entit

In [80]:
import pandas as pd
import asyncio
import nest_asyncio

# Allow nested use of asyncio in environments like Jupyter
nest_asyncio.apply()

raw_outputs = []
outputs = []

# Assuming you have an asynchronous function to run the model
async def process_content(content: str, idx: int, llm, parser):
    prompt = create_prompt(DEFAULT_BASE_PROMPT, parser, content)
    result = await run(llm, prompt, parser)
    raw_outputs.append(result)  # Store raw output for logging or debugging purposes

    output_map = {
        "article_id": idx,
    }

    if isinstance(result, CulturalHeritageSchema):
        output = output_map | result.dict()
        outputs.append(output)
    else:
        output_map["raw_llm_output"] = result
        outputs.append(output_map)

    return result

async def main():
    # Placeholder for the processed triplets
    processed_triplets = []

    for i in range(df.shape[0]):
        content = df["content"][i]
        result = await process_content(content, i, llm, parser)

        # Only continue if the result is of the expected type
        if isinstance(result, CulturalHeritageSchema):
            for triplet in result.triplets:
                processed_triplets.append({
                    "article_id": i,
                    "entity1": triplet.entity1.name,
                    "relation": triplet.relation.name,
                    "entity2": triplet.entity2.name
                })

    # Create DataFrame with columns entity1, relation, entity2
    result_df = pd.DataFrame(processed_triplets, columns=["article_id", "entity1", "relation", "entity2"])

    return result_df

# Function to run the async function in both environments
def run_main_loop():
    # Get or create the event loop
    try:
        # If there's already an event loop, use it
        loop = asyncio.get_running_loop()
    except RuntimeError:
        # If no event loop is running, create one
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)

    return loop.run_until_complete(main())

# Run the main async function
result_df = run_main_loop()
print(result_df)

/usr/local/lib/python3.10/dist-packages/pydantic/json_schema.py:334: RuntimeWarning: coroutine 'main' was never awaited
  mapping[key] = getattr(self, method_name)


    article_id               entity1                          relation  \
0            0           Marion True            RelationType.works_for   
1            0             Jiri Frel            RelationType.works_for   
2            0           Marion True             RelationType.has_role   
3            0             Jiri Frel             RelationType.has_role   
4            0  J. Paul Getty Museum    RelationType.has_possession_of   
..         ...                   ...                               ...   
63           2        Giacomo Medici  RelationType.legal_status_change   
64           2        Giacomo Medici  RelationType.legal_status_change   
65           2        Giacomo Medici  RelationType.legal_status_change   
66           2        Giacomo Medici             RelationType.has_role   
67           2        Giacomo Medici             RelationType.has_role   

                   entity2  
0     J. Paul Getty Museum  
1     J. Paul Getty Museum  
2   Curator of Antiquiti

In [83]:
result_df.to_csv('out.csv', index=True)